In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep
import re
from bs4 import BeautifulSoup as bs


my_options = webdriver.ChromeOptions()

#最大化視窗
my_options.add_argument("--start-maximized")
#開啟無痕模式    
my_options.add_argument("--incognito")
#禁用彈出攔截
my_options.add_argument("--disable-popup-blocking")
#取消通知
my_options.add_argument("--disable-notifications")

driver = webdriver.Chrome(options=my_options)

In [17]:
base_url = "https://www.gutenberg.org"
language_page = f"{base_url}/browse/languages/zh"

def openChrome():
    driver.get(language_page)

def contains_chinese(text):
    # 正则表达式，匹配中文字符范围
    chinese_regex = re.compile(r'[\u4e00-\u9fff]')
    # 搜索字符串中是否包含中文
    return bool(chinese_regex.search(text))

def getBookList():
    books = driver.find_elements(
        By.CSS_SELECTOR,'li.pgdbetext a'
    )

    booksList = []
    for i in books:
        bookTitle = i.get_attribute('innerText')
        if contains_chinese(bookTitle) :
            booksList.append(
                {
                    'title' : bookTitle,
                    'link' : i.get_attribute('href')
                }
            )
    
    return booksList

def openBooksPage(bookList):
    driver.get(bookList[0]['link'])

In [18]:
openChrome()
book = getBookList()
print(len(book))
print(book)

openBooksPage(book)

478
[{'title': '豆棚閒話', 'link': 'https://www.gutenberg.org/ebooks/25328'}, {'title': '戲中戲', 'link': 'https://www.gutenberg.org/ebooks/24225'}, {'title': '比目魚', 'link': 'https://www.gutenberg.org/ebooks/24185'}, {'title': '比目魚', 'link': 'https://www.gutenberg.org/ebooks/27119'}, {'title': '三字經', 'link': 'https://www.gutenberg.org/ebooks/12479'}, {'title': '山水情', 'link': 'https://www.gutenberg.org/ebooks/25146'}, {'title': '山海經', 'link': 'https://www.gutenberg.org/ebooks/25288'}, {'title': '施公案', 'link': 'https://www.gutenberg.org/ebooks/23825'}, {'title': '施公案', 'link': 'https://www.gutenberg.org/ebooks/25393'}, {'title': '易經', 'link': 'https://www.gutenberg.org/ebooks/25501'}, {'title': '木蘭奇女傳', 'link': 'https://www.gutenberg.org/ebooks/23938'}, {'title': '海公案', 'link': 'https://www.gutenberg.org/ebooks/54494'}, {'title': '燕丹子', 'link': 'https://www.gutenberg.org/ebooks/24068'}, {'title': '狄公案', 'link': 'https://www.gutenberg.org/ebooks/27686'}, {'title': '百家姓', 'link': 'https://www.gut